In [1]:
import numpy as np
import igl
import meshplot as mp

In [2]:
# Utility function to generate a tet grid
# n is a 3-tuple with the number of cell in every direction
# mmin/mmax are the grid bounding box corners

def tet_grid(n, mmin, mmax):
    nx = n[0]
    ny = n[1]
    nz = n[2]
    
    delta = mmax-mmin
    
    deltax = delta[0]/(nx-1)
    deltay = delta[1]/(ny-1)
    deltaz = delta[2]/(nz-1)
    
    T = np.zeros(((nx-1)*(ny-1)*(nz-1)*6, 4), dtype=np.int64)
    V = np.zeros((nx*ny*nz, 3))

    mapping = -np.ones((nx, ny, nz), dtype=np.int64)


    index = 0
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                mapping[i, j, k] = index
                V[index, :] = [i*deltax, j*deltay, k*deltaz]
                index += 1
    assert(index == V.shape[0])
    
    tets = np.array([
        [0,1,3,4],
        [5,2,6,7],
        [4,1,5,3],
        [4,3,7,5],
        [3,1,5,2],
        [2,3,7,5]
    ])
    
    index = 0
    for i in range(nx-1):
        for j in range(ny-1):
            for k in range(nz-1):
                indices = [
                    (i,   j,   k),
                    (i+1, j,   k),
                    (i+1, j+1, k),
                    (i,   j+1, k),

                    (i,   j,   k+1),
                    (i+1, j,   k+1),
                    (i+1, j+1, k+1),
                    (i,   j+1, k+1),
                ]
                
                for t in range(tets.shape[0]):
                    tmp = [mapping[indices[ii]] for ii in tets[t, :]]
                    T[index, :]=tmp
                    index += 1
                    
    assert(index == T.shape[0])
    
    V += mmin
    return V, T

# Reading point cloud

In [3]:
pi, v = igl.read_triangle_mesh("data/cat.off")
pi /= 10
ni = igl.per_vertex_normals(pi, v)
mp.plot(pi, shading={"point_size": 8})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

In [4]:
print(pi[1])  #Stampa il secondo punto
print(pi[1,1]) #Prende il secondo campo del secondo punto
print(v.shape)

[-0.1 -6.   1. ]
-6.0
(698, 3)


# Setting up  the constraints

In [5]:


#Function that retrieves the index of the closest point to 'point' in 'points'
#Use KD-Tree algorithm to find the nearest neighbor
from scipy.spatial import KDTree 
tree = KDTree(pi,leafsize = pi.shape[0] +1)  # leafsize is the number of points at which to switch to brute force

def find_closed_point(point,points):
    dist , ind = tree.query([point],k = 2) #k is the number of neighbors to return , and point is the array of points to query
    return ind[0,1] #Each entry gives the list of indices of neighbors of the corresponding point (prende il secondo)

In [12]:
# Add here the code to generate the additional points and constraints

def Constraints(pi,n):
    size = pi.shape[0] #366 points
    constrained_pi , constrained_v  = np.zeros((size*3 , 3)) , np.zeros((size*3))
    print(constrained_pi)
    print(constrained_v)
    print(constrained_v.shape)
    for i in range(size):
        eps = 0.01 * igl.bounding_box_diagonal(pi)# Fix an eps value 
        #for each point in the point cloud , add a costraint of the form f(pi)= di = o
        constrained_pi[i]= pi[i]
        constrained_v[i] = 0
        #for each point , compute p_i+N = p_i + eps * n_i
        #where n_i is the normalized normal at p_i.
        constrained[i+size] =  constrained[i] = eps * ni[i]
        #Check that p_i is the closest point to p_i+N
        #If not , halve eps and recompute p_i+N until this is the case
        #Then, add another constraint equation f(p_i+N) = d_i+N = eps


In [13]:
Constraints(pi,n= 20)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[0. 0. 0. ... 0. 0. 0.]
(1098,)


# MLS function

In [ ]:
# This is mockup code, generating the distance field from the surface of the unit sphere in analytic form
# The code is provided to show the desired visualization
# Use tet_grid to generate the grid appropriate to your data
# Generate function fx at all nodes of the grid by evaluating the MLS function

In [ ]:
# Parameters
bbox_min = np.array([-1., -1., -1.])
bbox_max = np.array([1., 1., 1.])
bbox_diag = np.linalg.norm(bbox_max - bbox_min)

n = 10

In [ ]:
# Generate grid n x n x n

x, T = tet_grid((n, n, n), bbox_min - 0.05 * bbox_diag, bbox_max + 0.05 * bbox_diag)

#Compute implicit sphere function
center = np.array([0., 0., 0.])
radius = 1
fx = np.linalg.norm(x-center, axis=1) - radius

In [ ]:
# Treshold fx to visualize inside outside

ind = np.zeros_like(fx)
ind[fx >= 0] = 1
ind[fx < 0] = -1
mp.plot(x, c=ind, shading={"point_size": 0.1,"width": 800, "height": 800})

# Marching to extract surface

In [ ]:
# Marcing tet to extract surface

sv, sf, _, _ = igl.marching_tets(x, T, fx, 0)
mp.plot(sv, sf, shading={"wireframe": True})